# Programmer des jeux avec SenseHat 

Voici les jeux
* jeu de Morpion (Hélènve, Victoria)
* jeu de Démineur (Mirko, Fabian)
* jeu de mémoire (Pierre, Jérémy)
* jeu de Tetris (Valentin, Hugo)
* jeu du 2048 (Michael, Massimo)
* jeu Puissance 4 (Térence, Niza)

## Introduction
Tout d'abord il est nécessaire d'importer du module `sense_hat` la classe `SenseHat`. Ceci permet d'instantier un objet `sense` qui va être utilisé pour commuiquer avec la carte. Nous tournons également l'affichage de 180 degrés.

In [120]:
from sense_hat import SenseHat
#from sense_emu import SenseHat
from time import sleep, time

sense = SenseHat()
sense.set_rotation(180)

### Afficher un message
La fonction `show_message` permet d'afficher un message.

In [99]:
sense.show_message('hello world')

Nous pouvons changer la couleur du texte, et la vitesse du défilement en utilisant des paramẗres optionels.

In [100]:
blue = (0, 0, 255)
sense.show_message('hello world', text_colour=blue, scroll_speed=0.2)

### Afficher des lettres
La fonction `show_letter` afficher une seule lettre. Voici les lettre de l'alphabet.

In [101]:
for c in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    sense.show_letter(c)
    sleep(0.2)

Ici on affiche les lettre de 0 à 9.

In [102]:
for i in range(10):
    sense.show_letter(str(i))
    sleep(0.2)

### Afficher des couleurs
Les couleurs sont représenté par les 3 composantes des couleurs primaires rouge, vert et blue (RGB, red-green-blue, en anglais). Chaque composante est exprimé par une entiers entre 0 et 255, 255 étant la luminosité maximum.
Nous définissons les couleurs de base comme tuples RGB.

In [40]:
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)
magenta = (255, 0, 255)
cyan = (0, 255, 255)
yellow = (255, 255, 0)
orange = (255, 128, 0)
white = (255, 255, 255)
black = (0, 0, 0)

Nous mettons les couleurs dans une liste et nous pouvons parcourir cette liste dans une boucle `for`.

In [103]:
colors = (red, magenta, blue, cyan, green, yellow, white, orange)
for col in colors:
    sense.clear(col)
    sleep(0.5)

## Contrôler des pixels
Les pixels sont addressés avec $x\in [0, ..., 7]$ et $y\in [0, ..., 7]$. Ceci allumes les 64 un après les autres.

In [104]:
sense.clear()
for x in range(8):
    for y in range(8):
        sense.set_pixel(x, y, blue)
        sleep(0.1)

Nous pouvons aussi créer de ligne avec différentes couleurs.

In [121]:
sense.clear()
for x in range(8):
    for y in range(8):
        sense.set_pixel(x, y, colors[x])
        sleep(0.1)

Nous pouvons aussi colorier chauqe deuxième rangés.

In [49]:
sense.clear()
for x in range(0, 8, 2):
    for y in range(8):
        sense.set_pixel(x, y, green)
        sleep(0.1)

Dans l'exemple suivant nous dessinons un **carrée** en utilisant un vecteur de déplacement $(dx, dy)$ pour calcuer la position $(x, y)$ du pixel.

In [106]:
sense.clear()
(x, y) = (0, 0)
dirs = ((1, 0), (0, 1), (-1, 0), (0, -1))
for (dx, dy) in dirs:
    for i in range(7):
        x = x + dx
        y = y + dy
        sense.set_pixel(x, y, red)
        sleep(0.1)

### Nombres aléatoires
La fonction `randint(a, b)` du module `random` retourne un entier aléatoire dans l'intervalle $[a, b]$.

In [108]:
from random import randint, choice
sense.clear()
for i in range(100):
    x = randint(0, 7)
    y = randint(0, 7)
    sense.set_pixel(x, y, green)
    sleep(0.1)

La fonction `choice(list)` retourne un élément aléatoire de la liste.

In [109]:
sense.clear()
for i in range(100):
    x = randint(0, 7)
    y = randint(0, 7)
    col = choice(colors)
    sense.set_pixel(x, y, col)
    sleep(0.1)

La fonction `choice(dirs)` retourne une direction aléatoire des 4 directions. Ceci est utilser pour créer un parcours aléatoire (**random walk**).

In [110]:
sense.clear()
(x, y) = (3, 3)
for i in range(100):
    (dx, dy) = choice(dirs)
    x = (x + dx) % 8
    y = (y + dy) % 8
    sense.set_pixel(x, y, blue)
    sleep(0.1)

### Bouger un pixel sur l'écran et tracer une ligne
Le program suivant décode les 4 directions de `event.directions` avec des `if` et `elif` pour déplacer un pixels rouge sur l'écran. Appuyper sur le bouton central termine le programme.
Il faut appuyer sur le bouton central pour terminer. 

In [131]:
sense.clear()
(x, y) = (2, 2)
running = True
while running:
    for event in sense.stick.get_events():
        if event.action == 'pressed':
            if event.direction == 'up':
                y = min(y+1, 7)
            elif event.direction == 'down':
                y = max(y-1, 0)
            if event.direction == 'left':
                x = min(x+1, 7)
            elif event.direction == 'right':
                x = max(x-1, 0)
            elif event.direction == 'middle':
                running = False
    sense.set_pixel(x, y, red)
sense.clear(green)

### Bouger un seul pixel
En utilisant un dictionaire avec les 4 directions $(dx, dy)$ ce programme peut être simplifié. Cette fois nous ne laissons pas de trace. Le utilise $(mod 8)$ pour calculer $(x, y)$ donc quand le curseur sort à gauche, il réapparit à droite, et vice-versa.

In [137]:
sense.clear()
(x, y) = (2, 2)
dirs = {'up':(0, 1), 'down':(0,-1), 'right':(1, 0), 'left':(-1, 0)}
running = True
while running:
    for event in sense.stick.get_events():
        if event.action == 'pressed':
            if event.direction in dirs:
                (dx, dy) = dirs[
                    event.direction]
            else:
                running = False
            x = (x + dx) % 8
            y = (y + dy) % 8
            sense.clear()
            sense.set_pixel(x, y, green)
sense.clear(blue)

### Mémoriser des pixels dans une liste
Maintenant nous modifions le programme pour laisser des pixels sur l'écran. Ces pixels sont mémorisé dans une liste `pixels`.

In [4]:
from sense_hat import SenseHat
sense = SenseHat()
sense.clear()
(x, y) = (2, 2)
dirs = {'up':(0, 1), 'down':(0,-1), 'right':(1, 0), 'left':(-1, 0)}
pixels = []
running = True
while len(pixels) < 10 :
    for event in sense.stick.get_events():
        if event.action == 'pressed':
            if event.direction in dirs:
                (dx, dy) = dirs[event.direction]
            else:
                pixels.append((x, y))
            x = (x + dx) % 8
            y = (y + dy) % 8
            sense.clear()
            for (a, b) in pixels:
                sense.set_pixel(a, b, red)
            sense.set_pixel(x, y, green)
sense.clear(blue)

KeyboardInterrupt: 

## Game of PONG
Le jeu de Pong est un des plus anciens jeux

In [118]:
# ball position
x, y   = (3, 3)
dx, dy = (1, 1)
bat_y = 3

draw_bat():
    pass

draw_ball():
    sense.draw_pixel(2, 2) 
    
running = True:
while running:
    sense.clear()
    draw_ball()
    draw_bat()
    sleep(0.2)

SyntaxError: invalid syntax (<ipython-input-118-5fcc962d82b3>, line 6)

## Créer un module

Changeons le dosser vers le dossiers des jeux.

In [1]:
%cd ../games/xxx

/home/pi/oc-2018/games/xxx


Toutes les 6 modules se trouvent ici.

In [5]:
%ls

connect4.py  game2048.py  games.py  mines.py  morpion.py  tetris.py


Importons le module **games**.

In [2]:
import games

module name = morpion
module name = game2048
module name = mines
module name = tetris
module name = connect4
module name = games


Affichons la documentation de ce module à l'aide de la commande **help**.

In [3]:
help(games)

Help on module games:

NAME
    games

DESCRIPTION
    File: games.py
    Author: Raphael Holzer
    Date: 27.01.2019
    
    This module imports 6 SenseHAT game modules :
    - morpion
    - game2048
    - mines
    - tetris
    - connect4
    - labyrinth

FUNCTIONS
    main()
        Present a question mark (?) and allow to choose a game.
        The left/right button increments/decrements the index number.
        The up button displays the game name.
        The middle button starts the selected game.

DATA
    BLUE = (0, 0, 255)
    GREEN = (0, 255, 0)
    RED = (255, 0, 0)

FILE
    /home/pi/oc-2018/games/xxx/games.py




Il est possible d'afficher de l'aide pour une fonction spécifique.

In [9]:
help(games.main)

Help on function main in module games:

main()
    Present a question mark (?) and allow to choose a game.
    The left/right button increments/decrements the index number.
    The up button displays the game name.
    The middle button starts the selected game.

